# Pandas Test - Real life exercise

In [1]:
import pandas as pd

For visualizations:

In [2]:
import cufflinks as cf; cf.go_offline()
import plotly_express as px

## About the data

The data you are about to load is coming from an Ad recommentation system of the company InBrain. InBrain is a adTech company and specalize in next-level ad recommendation.  
The company has two major products - one giving high quality recommendation while the other is giving an highly yielding recommendation. Inbrain custormers are sending requests, asking for either of the products. 

Once a week, the company is doing an internal quality assements, and sends a sample of its traffic requests to external annotation process, in which the annotators are trained to manually decided whether the recommended Ad is a succes or not. The data contains information about the ad (**ad_id**), the sampled week (**week_id**), the requester company (**requester**), the region of operation (**region**), the recommendation type (Quality/Yield, **recommendation_type**) and the recommendation provider algorithm (**rec_provider**). The annotators are marking whether the recommendation is meeting the Quality/Yield standards (**is_success**) and whether or not the recommendation is severly defected (**is_sever**)

See a sample below:

In [3]:
ad_recs_annotated = pd.read_csv('data/ad_annotations.csv')
ad_recs_annotated.sample(5)

,week_id,ad_id,requester,region,is_success,is_sever,recommendation_type,rec_provider
313184,2022-week_03,AD08R888TM,Extreme DA,US,yes,NaN,Yield,RNNY
142202,2021-week_42,AD097K8M8K,Extreme DA,US,yes,NaN,Yield,DNNY
175406,2021-week_45,AD00HH8VDC,Fry Multimedia,GB,yes,NaN,Qality,XGBQ
320124,2022-week_04,AD00LUH6XP,RelayFoods,GB,yes,NaN,Qality,DNNQ
163542,2021-week_44,AD09Q8S8Q8,Search Million Culture,GB,yes,NaN,Qality,DNNQ


Your job, as the new and only data scientist of the company, is to get familiar with the the data, show week-over-week trends and produce insightfull graphs as a preparation to a full blown BI dashboard.

## Questions

### Data Modifications

* Add a column with the sample year
* Add a column with the sample quarter (google how to)
* Add a new success column with values 1 and 0 for yes and no 
* Are there duplicated ads? To compac the data, remove duplicated ads and instead add an ad_count column **(pay attention, this is not trivial at all)**
* Are there any NaNs in the is_sever column? Count how many and create a new column with NaNs filled as False. Check that indeed the new column contaion no NaNs. 
* Capitlize (first letter only) the is_success column

**Solution 1**

In [4]:
ad_recs_annotated['year'] = ad_recs_annotated.week_id.apply(lambda x: x[:4])
ad_recs_annotated['year'] = ad_recs_annotated['year'].astype('int')
ad_recs_annotated.head(1)

,week_id,ad_id,requester,region,is_success,is_sever,recommendation_type,rec_provider,year
0,2021-week_33,AD0088VOS,Search Million Culture,DE,yes,NaN,Qality,DNNQ,2021


In [5]:
ad_recs_annotated['week'] = ad_recs_annotated['week_id'].str[-2:]
ad_recs_annotated['week'] = ad_recs_annotated['week'].astype('int')
ad_recs_annotated.head(1)

,week_id,ad_id,requester,region,is_success,is_sever,recommendation_type,rec_provider,year,week
0,2021-week_33,AD0088VOS,Search Million Culture,DE,yes,NaN,Qality,DNNQ,2021,33


**Solution 2**

In [6]:
wk=ad_recs_annotated['week'].astype('int')
ad_recs_annotated['quarter'] = ((wk-1) // 13) + 1
ad_recs_annotated.head(1)

,week_id,ad_id,requester,region,is_success,is_sever,recommendation_type,rec_provider,year,week,quarter
0,2021-week_33,AD0088VOS,Search Million Culture,DE,yes,NaN,Qality,DNNQ,2021,33,3


**Solution 3**

In [7]:
ad_recs_annotated['new_success'] = ad_recs_annotated['is_success'].apply(lambda x: 1 if x == 'yes' else 0)
ad_recs_annotated.sample(5)

,week_id,ad_id,requester,region,is_success,is_sever,recommendation_type,rec_provider,year,week,quarter,new_success
325622,2022-week_04,AD08WWTN6M,Extreme DA,US,yes,NaN,Yield,RNNY,2022,4,1,1
573,2021-week_33,AD07L88SD8,Search Million Culture,DE,yes,NaN,Qality,DNNQ,2021,33,3,1
5039,2021-week_33,AD00CHLV08,iDreamsky Technology,GB,no,True,Qality,DNNQ,2021,33,3,0
285279,2022-week_01,AD07K8WW8W,Extreme DA,US,yes,NaN,Yield,RNNY,2022,1,1,1
253443,2021-week_50,AD07YG8H8Y,RelayFoods,JP,yes,NaN,Qality,UserPopQ,2021,50,4,1


**Solution 4**

In [8]:
ad_recs_annotated.duplicated(subset='ad_id').sum()

73061

Conclusion: We can see that there are duplicates in ad_id column.

In [9]:
ad_recs_annotated['ad_count'] = 1
ad_recs_annotated.groupby(['ad_id']).ad_count.count().reset_index()

,ad_id,ad_count
0,AD00000088,1
1,AD000000WF,1
2,AD00000876,1
3,AD00000888,2
4,AD0000088C,1
...,...,...
300722,AD98980890,1
300723,AD98988688,1
300724,AD98988898,2
300725,AD99798888,1


In [10]:
#Check the results with an alternative code
no_duplicates = ad_recs_annotated.pivot_table(columns=['ad_id'], aggfunc='size',margins_name='ad_count')
print(no_duplicates)

ad_id
AD00000088    1
AD000000WF    1
AD00000876    1
AD00000888    2
AD0000088C    1
             ..
AD98980890    1
AD98988688    1
AD98988898    2
AD99798888    1
ADOO8X898E    2
Length: 300727, dtype: int64


**Solution 5**

In [11]:
ad_recs_annotated.is_sever.isnull().sum()

304195

Conclusion: Yes, there are NaN values in column is_server.

In [12]:
ad_recs_annotated['new_is_sever'] = ad_recs_annotated['is_sever'].fillna(False)

In [13]:
ad_recs_annotated.sample(5)

,week_id,ad_id,requester,region,is_success,is_sever,recommendation_type,rec_provider,year,week,quarter,new_success,ad_count,new_is_sever
50341,2021-week_36,AD00FXYTLI,Search Million Culture,IN,no,True,Qality,UserPopQ,2021,36,3,0,1,True
332082,2022-week_05,AD00NE8HF8,Fancy,GB,yes,NaN,Qality,DNNQ,2022,5,1,1,1,False
41958,2021-week_35,AD07N8J8H8,MoJoe Brewing Company,US,yes,NaN,Yield,RNNY,2021,35,3,1,1,False
33305,2021-week_35,AD08JS6YLQ,SOLOMO365,GB,yes,NaN,Qality,DNNQ,2021,35,3,1,1,False
289949,2022-week_02,AD08C88G8R,RelayFoods,GB,no,True,Qality,XGBQ,2022,2,1,0,1,True


In [14]:
ad_recs_annotated.new_is_sever.isnull().sum()

0

**Solution 6**

In [15]:
ad_recs_annotated['is_success'] = ad_recs_annotated['is_success'].str.capitalize()

In [16]:
ad_recs_annotated.sample(5)

,week_id,ad_id,requester,region,is_success,is_sever,recommendation_type,rec_provider,year,week,quarter,new_success,ad_count,new_is_sever
277770,2022-week_01,AD08D77XZX,Puentes Company,IN,Yes,NaN,Yield,UserPopY,2022,1,1,1,1,False
47922,2021-week_36,AD0088G8OP,RelayFoods,GB,Yes,NaN,Qality,DNNQ,2021,36,3,1,1,False
176971,2021-week_45,AD088N87Q8,RelayFoods,GB,Yes,NaN,Qality,DNNQ,2021,45,4,1,1,False
242536,2021-week_49,AD086S88QH,Fancy,US,Yes,NaN,Qality,ManualQ,2021,49,4,1,1,False
193672,2021-week_46,AD07Q8JHPX,MoJoe Brewing Company,IN,Yes,NaN,Yield,UserPopY,2021,46,4,1,1,False


### Subset selection

for each question, sub-select the data by using the conditional selection (`[]`) and the `.query` API. Use `.shape` on the subselection to obtain the result.  
For example:
`df.query('some_condition').shape`

1. How many requests are there in US region? 
1. How many **successful** requests are there in US region? 
1. How many **successful** requests are there in US region, on week 52?
1. How many requests where delivered by DDNQ, RRNY and UserPopQ together? 
1. How many requests where delivered by rule based providers? 
1. Select only the requests from 2021 - How many are there? 
1. Select the reqeusts from week 30 to week 40 - How many are there? 
1. Select all the data that comes from the most popular Ad
1. Select all the data that comes from the least popular provider
1. Select the data in which is_sever is None. How big is it? 
1. Select the data in which the requester is a 3 word company
1. Select the data in which the requester is a 3 word company, and the ad_id has the letter 6 in it
1. Select the data in which the requester is a 3 word company, and the multiplication of all the numbers in the ad_id is bigger than 30

**Solution 1**

In [17]:
ad_recs_annotated.query('region == "US"').shape[0]

103846

In [18]:
ad_recs_annotated.loc[ad_recs_annotated.region == 'US'].shape[0]

103846

**Solution 2**

In [19]:
ad_recs_annotated.loc[((ad_recs_annotated['region'] == 'US') & (ad_recs_annotated['new_success'] == 1))] .shape[0]

88918

In [20]:
ad_recs_annotated.query('region == "US" & new_success == 1').shape[0]

88918

**Solution 3**

In [21]:
ad_recs_annotated.loc[((ad_recs_annotated['region'] == 'US') & (ad_recs_annotated['new_success'] == 1) &
                      (ad_recs_annotated['week'] == 52))] .shape[0]

3342

In [22]:
ad_recs_annotated.query('region == "US" & new_success == 1 & week == 52').shape[0]

3342

**Solution 4**

In [23]:
ad_recs_annotated.loc[((ad_recs_annotated['rec_provider'] == 'DDNQ') | (ad_recs_annotated['rec_provider'] == 'RRNY')| 
                       (ad_recs_annotated['rec_provider'] == 'UserPopQ'))].shape[0]

69937

In [24]:
ad_recs_annotated.query('rec_provider == "DDNQ" | rec_provider == "RRNY" | rec_provider == "UserPopQ"').shape[0]

69937

**Solution 5**

In [25]:
ad_recs_annotated.loc[((ad_recs_annotated['rec_provider'] == 'RuleBasedY'))].shape[0]

28154

In [27]:
ad_recs_annotated.query('rec_provider == "RuleBasedY"').shape[0]

28154

**Solution 6**

In [26]:
ad_recs_annotated.loc[((ad_recs_annotated['year'] >= 2021))].shape[0]

373788

In [27]:
ad_recs_annotated.query('year >= 2021').shape[0]

373788

**Solution 7**

In [28]:
ad_recs_annotated.loc[((ad_recs_annotated['week'] >= 30) &
                       (ad_recs_annotated['week'] <= 40))].shape[0]

115051

In [29]:
ad_recs_annotated.query('week >= 30 & week <= 40').shape[0]

115051

**Solution 8**

In [30]:
most_popular_ad = ad_recs_annotated['ad_id'].value_counts().idxmax() # I've assumed that the most popular ad is the ad with the higher frequency
print(most_popular_ad)

AD07PFFMP9


In [31]:
print(ad_recs_annotated['ad_id'].value_counts()) # In order to prove that I get the same result as above

AD07PFFMP9    247
AD098SWYF6    239
AD08C8RR8J    138
AD0886VY87     99
AD08888888     96
             ... 
AD07Y8JSHJ      1
AD0888J7AO      1
AD087G7SHR      1
AD086QCZGJ      1
AD08FWNFDO      1
Name: ad_id, Length: 300727, dtype: int64


In [32]:
ad_recs_annotated.query('ad_id == @most_popular_ad').shape[0]

247

In [33]:
ad_recs_annotated.loc[ad_recs_annotated.ad_id == most_popular_ad].shape[0]

247

**Solution 9**

In [34]:
least_popular_provider = ad_recs_annotated['rec_provider'].value_counts().index[-1]
print(least_popular_provider)

UserPopSelectionY


In [35]:
print(ad_recs_annotated['rec_provider'].value_counts()) #in order to prove that i get the same result as above

DNNQ                 117424
UserPopQ              69937
DNNY                  45116
UserPopY              38600
RuleBasedY            28154
RNNQ                  20983
ManualQ               13844
RNNY                  12732
XGBQ                  12250
XGBY                   4276
BooksY                 4150
UserPopSelectionQ      2417
ManualY                1982
BooksQ                 1720
RuleBased               182
UserPopSelectionY        21
Name: rec_provider, dtype: int64


In [36]:
ad_recs_annotated.query('rec_provider == @least_popular_provider').shape[0]

21

In [37]:
ad_recs_annotated.loc[ad_recs_annotated.rec_provider == least_popular_provider].shape[0]

21

**Solution 10**

In [38]:
ad_recs_annotated['is_sever'].isna().sum()

304195

**Solution 11**

In [39]:
ad_recs_annotated.loc[(ad_recs_annotated['requester'].str.count(' ') + 1) == 3].shape[0]

118141

**Solution 12**

In [40]:
ad_recs_annotated.loc[( (ad_recs_annotated['requester'].str.count(' ') + 1) == 3 ) & 
                      (ad_recs_annotated['ad_id'].str.contains('6'))].shape[0]

24197

**Solution 13**

In [41]:
import re
def find_number(text):
    num = re.findall(r'[0-9]+',text)
    return " ".join(num)

number=ad_recs_annotated['ad_id'].apply(lambda x: find_number(x))
number.str.strip()

0             0088
1             07 8
2           08 6 9
3         89608808
4             07 6
            ...   
373783        08 8
373784        07 8
373785      096 88
373786          08
373787        00 8
Name: ad_id, Length: 373788, dtype: object

In [43]:
def remove(string):
    return string.replace(" ", "")
number_no_spaces = number.apply(lambda x: remove(x))

In [44]:
number_no_spaces = number_no_spaces.astype('int')

In [45]:
def no_mult(x):
    sum = 1

    while x > 0:
        d = x%10
        x = x//10
        sum *= d
    return sum

numb_mult=number_no_spaces.apply(lambda x: no_mult(x))

In [46]:
ad_recs_annotated.loc[( (ad_recs_annotated['requester'].str.count(' ') + 1) == 3 ) & 
                      (numb_mult > 30)]

,week_id,ad_id,requester,region,is_success,is_sever,recommendation_type,rec_provider,year,week,quarter,new_success,ad_count,new_is_sever
0,2021-week_33,AD0088VOS,Search Million Culture,DE,Yes,NaN,Qality,DNNQ,2021,33,3,1,1,False
1,2021-week_33,AD07KYS8JM,MoJoe Brewing Company,DE,Yes,NaN,Yield,DNNY,2021,33,3,1,1,False
2,2021-week_33,AD08PDP6Y9,Search Million Culture,DE,Yes,NaN,Qality,DNNQ,2021,33,3,1,1,False
4,2021-week_33,AD07CMVHP6,Search Million Culture,DE,Yes,NaN,Qality,DNNQ,2021,33,3,1,1,False
7,2021-week_33,AD07NMJ88F,Search Million Culture,DE,Yes,NaN,Qality,DNNQ,2021,33,3,1,1,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
373663,2022-week_52,AD09G98VJZ,MoJoe Brewing Company,US,Yes,NaN,Yield,RNNY,2022,52,4,1,1,False
373689,2022-week_52,AD078WDG8W,MoJoe Brewing Company,US,Yes,NaN,Yield,RNNY,2022,52,4,1,1,False
373691,2022-week_52,AD08M86WW8,MoJoe Brewing Company,US,Yes,NaN,Yield,DNNY,2022,52,4,1,1,False
373722,2022-week_52,AD09C8X6VY,Search Million Culture,US,Yes,NaN,Qality,DNNQ,2022,52,4,1,1,False


### Analysis

1. How many weeks available in the data? Are there any missing weeks? 
1. How many regions available in the data? Are there any missing weeks per region? 
1. How many ads are being sent to annotation in each region per week?
1. How many None values are there in is_sever column? 
1. Are ads being sent more than one time in a given week? what is the ad_id that was sent the most in a given week? (e.g. ad X where sent 50 times in week Y)
1. Are there ads that are being sent in more than one week?
groupby ad_id week idmax
   1. Which is the ad that was sent in most amount of weeks (e.g. ad X where sent in Y different weeks)
   2. What are the weeks that the ad from above was sent in? 
   3. Is there an Ad that was succsefull in one week, but not succesfful in a different week? Show one.
1. When is_sever is None, what is the number of successful requests? What is the number of non-successful requests? What do you learn from it about the reason for Nones in the first place? 
1. Per each region, What is the Quality/Yield traffic proportion WoW? week over week, over time
Proportion between quality and yield. needs to be sum = 1, per week

1. How many different reqeusters are there? 
1. Which are the top 5 requesters per region? 
1. Which are the different rec_proivders?
1. Are there different rec providers per region? 
1. Are there different rec providers per rec type?
1. What are the notation rules distinguishing between quality vs yielding providers?
meafien a shmot shel a provider, mistaim v "q" = quality
kol ma sh magia mi provider q u mi quiality


1. Which is the most successful region of operation? 
1. Which is the most successful rec provider per each region? 
1. Present a table with a success rate, sever defects rate and CI (for each metric) per region
1. Present a table with a success rate, sever defects rate and CI (for each metric) per rec provider
1. Present a table with a success rate, sever defects rate and CI (for each metric) per region and rec provider combinations
1. Present a table with a success rate, sever defects rate and CI (for each metric) per rec type (Q/Y)
1. Present a table with a success rate, sever defects rate and CI (for each metric) per rec type and region
1. Present WoW table/graph with success rate and CI (see image below) per region
1. Present WoW table/graph with success rate and CI (see image below) per region and rec type
1. Present WoW table/graph with success rate and CI (see image below) per region and rec provider
1. Which are the requester to which we succeed the most?
1. Which are the requester to which we produce the most critical defects?
1. What is the overall success rate trend over time? 
1. What is the overall sever defect rate trend over time? 
1. Preset a WoW graph showing the number of requests per customer in each region (hint: [use stacked bars](https://plotly.com/python/bar-charts/)), from it:
  1. Identify major traffic shape changes (addition/removal) of major clients
1. Preset a WoW graph showing the **requests proportion** per customer in each region (hint: [use stacked bars](https://plotly.com/python/bar-charts/)), from it:
  1. Identify major traffic shape changes (addition/removal) of major clients


**Solution 1**

In [47]:
ad_recs_annotated.value_counts('week').count()

26

In [48]:
ad_recs_annotated['week'].isnull().count()

373788

**Solution 2**

In [49]:
ad_recs_annotated.value_counts('region').count()

5

**Solution 3**

In [50]:
ad_recs_annotated[["region","week"]].value_counts().reset_index(name ='Total Ads')

,region,week,Total Ads
0,US,2,4016
1,US,48,4010
2,US,1,4009
3,US,34,4006
4,US,42,4004
...,...,...,...
124,JP,4,1586
125,JP,35,1580
126,JP,3,1578
127,JP,49,1577


**Solution 4**

In [51]:
ad_recs_annotated['is_sever'].isnull().count()

373788

**Solution 5**

In [52]:
ad_recs_annotated[["ad_id","week"]].value_counts() > 1

ad_id       week
AD098SWYF6  40       True
AD08C8RR8J  47       True
AD07QS8VCL  47       True
AD00IGCC8G  47       True
AD098C6SNV  47       True
                    ...  
AD07GRVV8L  47      False
AD07GRVT8R  37      False
AD07GRTNSP  49      False
AD07GRTNPJ  37      False
ADOO8X898E  50      False
Length: 358763, dtype: bool

In [53]:
ad_recs_annotated[["ad_id","week"]].value_counts().idxmax()

('AD098SWYF6', 40)

**Solution 6**

In [ ]:
Are there ads that are being sent in more than one week?
Which is the ad that was sent in most amount of weeks (e.g. ad X where sent in Y different weeks)
What are the weeks that the ad from above was sent in?
Is there an Ad that was succsefull in one week, but not succesfful in a different week? Show one.

In [54]:
sol6_df = ad_recs_annotated.groupby(['ad_id','week'])['week'].count()

In [55]:
sol6_df_sorted = sol6_df.groupby(level=[0, 0]).transform("count").reset_index(name ='Total Amount').sort_values(by='Total Amount')

In [56]:
sol6_df_sorted

,ad_id,week,Total Amount
0,AD00000088,37,1
226830,AD089GRKZT,34,1
226829,AD089GRKDJ,49,1
226826,AD089GRF68,35,1
226825,AD089GR8YY,34,1
...,...,...,...
104890,AD079VP6DH,48,26
104891,AD079VP6DH,49,26
104892,AD079VP6DH,50,26
104882,AD079VP6DH,40,26


Conclusion: We can see above that there are ads that were sent in more than 1 week.

In [57]:
ad_most_weeks = sol6_df_sorted[['ad_id','Total Amount']].value_counts().idxmax()
# In order to calculate the ad that was sent in the most number of weeks
ad_most_weeks

('AD07PFFMP9', 26)

In [58]:
most_sold_add = ad_most_weeks[0]
print(most_sold_add) # Ad that was sent in the most number of weeks

AD07PFFMP9


In [59]:
most_sold_df = sol6_df_sorted.query('ad_id == @most_sold_add')
list(most_sold_df['week']) # List of the weeks from the most sold ad

[5,
 2,
 3,
 51,
 50,
 49,
 48,
 47,
 46,
 45,
 44,
 52,
 43,
 41,
 40,
 39,
 38,
 37,
 36,
 35,
 34,
 33,
 4,
 42,
 1,
 6]

In [100]:
ad_recs_annotated.groupby(['ad_id','week'])['week'].count()

ad_id       week
AD00000088  37      1
AD000000WF  3       1
AD00000876  34      1
AD00000888  2       1
            43      1
                   ..
AD98988898  5       1
            43      1
AD99798888  42      1
ADOO8X898E  41      1
            50      1
Name: week, Length: 358763, dtype: int64

In [107]:
df_new = ad_recs_annotated[['ad_id','week','new_success']]
df_new

,ad_id,week,new_success
0,AD0088VOS,33,1
1,AD07KYS8JM,33,1
2,AD08PDP6Y9,33,1
3,AD89608808,33,1
4,AD07CMVHP6,33,1
...,...,...,...
373783,AD08KQ8GDG,52,1
373784,AD07C8XSMN,52,1
373785,AD096LRR88,52,1
373786,AD08FWNFDO,52,1


In [127]:
res = df_new.groupby(['ad_id','week'])['new_success'].agg(['count','sum'])
res

count  sum
ad_id      week            
AD00000088 37        1    1
AD000000WF 3         1    1
AD00000876 34        1    1
AD00000888 2         1    1
           43        1    1
...                ...  ...
AD98988898 5         1    1
           43        1    1
AD99798888 42        1    0
ADOO8X898E 41        1    1
           50        1    1

[358763 rows x 2 columns]

In [132]:
res2= res.groupby('ad_id')[['count','sum']].sum()
res2

,count,sum
ad_id,,
AD00000088,1,1
AD000000WF,1,1
AD00000876,1,1
AD00000888,2,2
AD0000088C,1,1
...,...,...
AD98980890,1,1
AD98988688,1,1
AD98988898,2,2


In [137]:
res2['check'] = res2['count'] - res2['sum']

In [148]:
res2.loc[((res2['check'] != 0) & (res2['count'] > 1) & (res2['sum'] > 0))]

,count,sum,check
ad_id,,,
AD00006IDK,2,1,1
AD00006IEI,4,3,1
AD00007F8Q,4,3,1
AD00007K6N,2,1,1
AD00008808,5,4,1
...,...,...,...
AD98888878,2,1,1
AD98888888,8,7,1
AD98889888,3,1,2


In [151]:
ad_recs_annotated.loc[ad_recs_annotated.ad_id == 'AD98888878'] # Example taken from the query above in order to prove that it works

,week_id,ad_id,requester,region,is_success,is_sever,recommendation_type,rec_provider,year,week,quarter,new_success,ad_count,new_is_sever
21446,2021-week_34,AD98888878,Puentes Company,IN,Yes,NaN,Qality,UserPopQ,2021,34,3,1,1,False
165804,2021-week_44,AD98888878,Search Million Culture,IN,No,False,Qality,UserPopQ,2021,44,4,0,1,False


**Solution 7**

In [66]:
ad_recs_annotated.loc[((ad_recs_annotated['is_sever'].isnull()) & (ad_recs_annotated['new_success'] == 1))].shape[0]

304189

In [67]:
ad_recs_annotated.loc[((ad_recs_annotated['is_sever'].isnull()) & (ad_recs_annotated['new_success'] == 0))].shape[0]

6

Conclusion:  'None" is_sever values are in fact 'False' entries, ie the recommendation is not severly defected.

**Solution 8**

In [68]:
sol8_df = ad_recs_annotated.groupby(['region','week','new_success'])['new_success'].count() #.reset_index(name='counts')

In [69]:
sol8_df / sol8_df.groupby(level=[0, 1]).transform("sum")

region  week  new_success
DE      1     0              0.179017
              1              0.820983
        2     0              0.158730
              1              0.841270
        3     0              0.167788
                               ...   
US      50    1              0.847657
        51    0              0.176927
              1              0.823073
        52    0              0.164082
              1              0.835918
Name: new_success, Length: 258, dtype: float64

**Solution 9**

In [71]:
ad_recs_annotated.requester.nunique()

36

**Solution 10**

In [72]:
ad_recs_annotated.groupby(['region','requester'])['region'].count().sort_values(ascending=False).head(5)

region  requester             
US      Extreme DA                57215
IN      Puentes Company           29427
GB      Search Million Culture    28002
JP      RelayFoods                24574
DE      Search Million Culture    23657
Name: region, dtype: int64

**Solution 11**

In [73]:
print(ad_recs_annotated.groupby('rec_provider').groups.keys())

dict_keys(['BooksQ', 'BooksY', 'DNNQ', 'DNNY', 'ManualQ', 'ManualY', 'RNNQ', 'RNNY', 'RuleBased', 'RuleBasedY', 'UserPopQ', 'UserPopSelectionQ', 'UserPopSelectionY', 'UserPopY', 'XGBQ', 'XGBY'])


**Solution 12**

In [74]:
ad_recs_annotated.groupby(['region','rec_provider'])['region'].count().reset_index(name='Total')

,region,rec_provider,Total
0,DE,BooksQ,1353
1,DE,BooksY,1257
2,DE,DNNQ,32433
3,DE,DNNY,5417
4,DE,ManualQ,968
5,DE,ManualY,605
6,DE,RuleBasedY,1614
7,DE,UserPopQ,15179
8,DE,UserPopY,3351
9,GB,DNNQ,39675


**Solution 13**

In [75]:
ad_recs_annotated.groupby(['recommendation_type','rec_provider'])['recommendation_type'].count().reset_index(name='Total')

,recommendation_type,rec_provider,Total
0,Qality,BooksQ,1720
1,Qality,DNNQ,117424
2,Qality,ManualQ,10220
3,Qality,ManualY,1439
4,Qality,RNNQ,20983
5,Qality,RuleBased,182
6,Qality,UserPopQ,69937
7,Qality,UserPopSelectionQ,2417
8,Qality,XGBQ,12250
9,Yield,BooksY,4150


**Solution 14**

What are the notation rules distinguishing between quality vs yielding providers?
meafien a shmot shel a provider, mistaim v "q" = quality
kol ma sh magia mi provider q u mi quiality

In [101]:
ad_recs_annotated.groupby('rec_provider')['rec_provider'].count()

rec_provider
BooksQ                 1720
BooksY                 4150
DNNQ                 117424
DNNY                  45116
ManualQ               13844
ManualY                1982
RNNQ                  20983
RNNY                  12732
RuleBased               182
RuleBasedY            28154
UserPopQ              69937
UserPopSelectionQ      2417
UserPopSelectionY        21
UserPopY              38600
XGBQ                  12250
XGBY                   4276
Name: rec_provider, dtype: int64

In [ ]:
ad_recs_annotated.groupby('recommendation_type')['recommendation_type'].count()

In [85]:
step1 = ad_recs_annotated['recommendation_type'].str[0:1]
step1.str.upper()

0         Q
1         Y
2         Q
3         Q
4         Q
         ..
373783    Y
373784    Q
373785    Y
373786    Q
373787    Q
Name: recommendation_type, Length: 373788, dtype: object

In [86]:
step2 = ad_recs_annotated['rec_provider'].str[-1:]
step2.str.upper()

0         Q
1         Y
2         Q
3         Q
4         Q
         ..
373783    Y
373784    Q
373785    Y
373786    Q
373787    Q
Name: rec_provider, Length: 373788, dtype: object

In [94]:
step3 = (step1 == step2)
step3

0         True
1         True
2         True
3         True
4         True
          ... 
373783    True
373784    True
373785    True
373786    True
373787    True
Length: 373788, dtype: bool

In [95]:
ad_recs_annotated['check_notation'] = step3
ad_recs_annotated.sample(5)

,week_id,ad_id,requester,region,is_success,is_sever,recommendation_type,rec_provider,year,week,quarter,new_success,ad_count,new_is_sever,check_notation
96146,2021-week_39,AD07JKHNC8,RelayFoods,JP,Yes,NaN,Qality,DNNQ,2021,39,3,1,1,False,True
291960,2022-week_02,AD08988V9Q,Puentes Company,IN,Yes,NaN,Yield,UserPopY,2022,2,1,1,1,False,True
142431,2021-week_42,AD088DC88P,Extreme DA,US,Yes,NaN,Yield,DNNY,2021,42,4,1,1,False,True
212503,2021-week_47,AD088WFOFZ,Extreme DA,US,Yes,NaN,Yield,DNNY,2021,47,4,1,1,False,True
92046,2021-week_39,AD088TNYTM,LocalVox Media,IN,Yes,NaN,Yield,UserPopY,2021,39,3,1,1,False,True


In [ ]:
ad_recs_annotated['check_notation'].groupby

In [97]:
ad_recs_annotated.groupby('check_notation')['check_notation'].count().reset_index(name='Total')

,check_notation,Total
0,False,5245
1,True,368543


In [102]:
ad_recs_annotated.loc[ad_recs_annotated.check_notation == False].head(5)

,week_id,ad_id,requester,region,is_success,is_sever,recommendation_type,rec_provider,year,week,quarter,new_success,ad_count,new_is_sever,check_notation
24,2021-week_33,AD07SNPKX8,Search Million Culture,DE,Yes,NaN,Qality,ManualY,2021,33,3,1,1,False,False
313,2021-week_33,AD07PHPXHQ,iDreamsky Technology,DE,Yes,NaN,Qality,ManualY,2021,33,3,1,1,False,False
314,2021-week_33,AD07PHPXHQ,Search Million Culture,DE,Yes,NaN,Qality,ManualY,2021,33,3,1,1,False,False
315,2021-week_33,AD07PHPXHQ,iDreamsky Technology,DE,Yes,NaN,Qality,ManualY,2021,33,3,1,1,False,False
316,2021-week_33,AD07PHPXHQ,Search Million Culture,DE,Yes,NaN,Qality,ManualY,2021,33,3,1,1,False,False


Conclusion: In the same query we could see that there is a provider called "RuleBased" which doesn't have an indication of the recommendation type (Qality/Yield).
Also the above query shoes that there are 5245 entries where the recommendation type doesn't seem to match re indication in rec_provider name.
For example index 24, the recommendation_type is "Qality" but the last letter of rec_provider, ManualY, is indeed "Y".
This should be further investigated in order to check if the data is wrong.

**Solution 15**

In [105]:
success_set = ad_recs_annotated.query('new_success == 1')
success_set[["region","new_success"]].value_counts().idxmax()

('US', 1)

In [106]:
success_set.groupby(['region'])['new_success'].count() # in order to check the result

region
DE    50624
GB    69509
IN    63355
JP    31800
US    88918
Name: new_success, dtype: int64

Analysis Bonus questions:
1. Compute the per region success rate and CI in trailing weeks, Say, 4 weeks instead of 1 week - to allow for smoother estimations and smaller CIs

### Merges and joins

The Wow samples and annotations task were sent and recieve seperatly, per each week. The dataset you were working on was constructed from those files. You can see the files under the `data/weekly` folder, here are the first 10 files:

In [ ]:
import os
weekly_files = os.listdir('data/weekly/')
sorted(weekly_files)[:10]

Your task is to reconstruct the dataset above. 

### Visualizations

Produce the following success rate graph per region:

Produce the following requestser proportion graph:

## Appendix

Code to create weekly files:

In [ ]:
# for l in ad_recs_annotated.region.unique():
#     for w in ad_recs_annotated.week_id.unique():
#         w_id = w.split('_')[1]
#         y = w.split('-')[0]
#         query = f'week_id == "{w}" and region == "{l}"'
#         sample_f_name = f'{y}_{w_id}_{l}__Sample.csv'
#         ad_recs_annotated.query(query)[['week_id','ad_id','requester','region','recommendation_type', 'rec_provider']].to_csv(f'data/weekly/{sample_f_name}', index=False)
#         for s in ad_recs_annotated.recommendation_type.unique():
#             ann_f_name = f'{y}_{w_id}_{l}_{s}_annotation_result.csv'
#             query = f'week_id == "{w}" and region == "{l}" and recommendation_type == "{s}"'
#             sss = ad_recs_annotated.query(query)
#             if sss.empty:
#                 continue            
#             sss[['region','ad_id','is_success','is_sever']].drop_duplicates(subset=['ad_id']).to_csv(f'data/weekly/{ann_f_name}', index=False)